![IITIS](pictures/logoIITISduze.png)

# Introduction to Ising Model

The Ising model has begun its life as a model of ferromagnetism. Ernst Ising introduced it in the early 1920s, and it has since become a fundamental model for understanding phase transitions and emergent properties in various physical systems. In this course, we will focus on one problem related to the Ising model: finding its ground state (i.e., the state with the lowest energy) and a low energy spectrum. As will be shown later in the course, the Ising model is significant not only in physics but also influences fields such as finance, social dynamics, and optimization in computer science.




## Definition
Let us consider a vector of binary variables $\bm{s} = (s_1, s_2, \ldots, s_N)$. These variables are called **spin** and can be in one of two states: up (+1) or down (-1), formally $s_i = \pm 1$. Spins are arranged on some simple graph $G$, which describes interactions between them. The edges of $G$ (denoted $E(G)$) determine which spin interacts with which. The strength of the **interaction** (or **coupling**) between $s_i$ and $s_j$ is denoted by $J_{ij} \in \mathbb{R}$. Additionally, each spin is subjected to the **external magnetic field** (or simply **bias**) $h_i \in \mathbb{R}$. The energy of the system is described by the following Hamiltonian:
$$
H(\bm{s}) = \sum_{(i, j) \in E(G)} J_{ij} s_i s_j + \sum_{i=1}^{N} h_i s_i
$$

Additionaly each realization of $\bm{s}$ is called **state** or **configuration**. Our goal is to find $\bm{s}$ that minimizes $H(\bm{s})$. Such state is called a **ground state**, that is state with the lowest posible energy. If more than one ground state exists, they are said to be **degenerate**.

![example](pictures/example_new.png)

From a computing standpoint, looking at the Ising model in terms of linear algebra may be easier. We look into the adjacency matrix of graph $G$ and treat each $J_{ij}$ as weight assigned to the edge. As $G$ is undirected, $J_{ij} = J_{ji}$. To avoid needles work, we can treat this matrix as an upper-triangular matrix and only care for $J_{ij}$ where $ i < j$. It is customary to denote this matrix by $\bm{J}$. Additionally, we can order values $h_i$ into the vector $\bm{h}$ Then the equation for $H(\bm{s})$ is equal:
$$
H(\bm{s}) = \bm{s}\bm{J}\bm{s}^{T} + \bm{h}\bm{s}^{T}
$$

### Physical interpretation

Traditionally, in physic Ising model is described on $d$-dimentional lattice (graph $G$ in our definition). In the model of ferromagnetism, Ising original study, one may think of the lattice sites as being occupied by atoms of a magnetic material. Each atom has a magnetic moment (spin) which is allowed to point either "up" or "down." That way, if all sites point "up" or "down" we have a ferromagnet. On the other hand, if all neighboring sites point in oposite direction we will have anitferromagnet, without (or with vanishing) magnetisation. In this interpretation $h_i$ represents external magnetic field which forces magnetic moment into particular direction. Interaction between spins $J_i$ may be ferromagnetic if it promotes both sites pointing in the same direction., otherwise it is called antiferromagnetic. State in which spins are randomly pointing up or down is often called **spin glass**.



## Finding the ground state - Brute Force

To find the ground state, one might initially think to compute the energies of every possible spin configuration and simply choose the one with the lowest energy. This aproach is called **Brute Force** or **Exhaustive Search**. Let's consider a complete graph with three spins as an example.

In [10]:
import numpy as np
from itertools import product
from math import inf

# define Ising model (both ways)
h = {1: 0.5, 2: -1, 3: 0.75}
J = {(1, 2): 1, (1, 3): 0.5, (2, 3): -0.33}

h_vect = np.array([0.5, -1, 0.75])
J_matrix = np.array([[0, 1, 0.5],[0, 0, -0.33],[0, 0, 0]])

In this example we will use convienient tool from the `itertools` package - `product`. In python an iterator is an object that can be iterated upon, meaning that you can traverse through all the values. The `product` is an iterator that goes throught all values of cartesian product of given set. The `repeat` parameter specifies how many times we take the cartesian product. In this example we have $\{-1, 1\}^3$. For 100 spins it would be $\{-1, 1\}^100$.

In [6]:
# Calulate direcly from first definition
best_energy = inf
best_state = {}
for configuration_vector in product([-1, 1], repeat=3):
    energy = 0
    configuration = {i: configuration_vector[i-1] for i in range(1, 4)} # change vector into dictionary
    for (v1, v2), coupling_strength in J.items():
        energy += coupling_strength * configuration[v1] * configuration[v2]
    for v, bias in h.items():
        energy += bias * configuration[v]
    if energy < best_energy:
        best_energy = energy
        best_state = configuration

print(best_state)
print(best_energy)


{1: -1, 2: 1, 3: 1}
-2.58


In [12]:
# Calculate using vectors and matrices
best_energy = inf
best_state = []
for configuration_vector in product([-1, 1], repeat=3):
    configuration = np.array(configuration_vector)
    energy = np.dot(configuration.T, np.dot(J_matrix, configuration)) + np.dot(h_vect, configuration)
    if energy < best_energy:
        best_energy = energy
        best_state = configuration

print(best_state)
print(best_energy)

[-1  1  1]
-2.58


However, it is easy to see that the number of operations required scales exponentially as $2^N$. This exponential growth quickly makes the task unmanageable. For instance, even with just 100 spins, this brute force algorithm becomes practically impossible to execute.

### Certification of solutions
While some cases are exacly solvable (for example 1d ising chain), the exhaustive search method is the only known algorithm that **guarantees** finding the ground statefor arbitrary ising model. Additionally, it is only know method to certify given ground state. That is, check if found ground state is really state with the lowest energy. In general, finding the ground state of arbitrary ising model is belivied to be a NP-hard problem.

In the latter part of the course we will look into some heuristic, physically inspired algorithms for solving this problem, namely simulated annealing, parrarel tempering, simulated bifurcation and Branch & Bound.


# Quantum Formulation

## Clasical Ising model

The Ising model hamiltonian may be formulated using Pauli matrices. As reminder, the Pauli $Z$ matrix $\sigma^{Z}$ is defined as:

$$\sigma^{Z} = \begin{bmatrix} 
1 & 0 \\ 
0 & -1 
\end{bmatrix}  $$
and Pauli $X$ matrix $\sigma^{x}$ is:

$$\sigma^{X} = \begin{bmatrix} 
0 & 1 \\ 
1 & 0 
\end{bmatrix}  $$

It is worth noting that both $\sigma^{Z}$ amd $\sigma^{X}$ has eigenvalues $1$ and $-1$. Coresponding eigenvectors are $|0\rangle$ and $|1\rangle$ for $\sigma^{Z}$ and $|+\rangle = \frac{|0\rangle + |1\rangle}{\sqrt{2}}$ and $|-\rangle = \frac{|0\rangle - |1\rangle}{\sqrt{2}}$ for $\sigma^{X}$, and  Additionaly, by telling that matrix $\sigma_i^{\alpha}$, $\alpha \in \{X, Z\}$ acts on spin $i$ we mean: 

$$
\sigma_i^{\alpha} = \underbrace{I  \otimes \ldots \otimes I}_{\text{$i-1$~times}} \otimes \underbrace{\sigma^{\alpha}}_{\text{$i$~-th place}} \otimes \underbrace{ I \otimes \ldots \otimes I}_{\text{$N-i$~times}}
$$

Then the Ising Hamiltonian is defined as follow:

$$
H(\sigma) = \sum_{i<j} J_{ij} \sigma^{Z}_i \sigma^Z_j + \sum_{i}^{N} h_i \sigma^Z_i 
$$

where $\bm{J}$ is an uper-triangular matrix describing interactions between spins and $\bm{h}$ is a vector of external magnetic fields. In literature one may find notation $\sigma_1$ for $\sigma^{X}$ and $\sigma_3$ for $\sigma^{Z}$.

## Transverse field Ising model
The Ising model in a transverse field can be described by the Hamiltonian

$$
H(\sigma) = \Gamma \sum_i^{N} \sigma_i^{X} + \sum_{i<j} J_{ij} \sigma^{Z}_i \sigma^Z_j
$$

where $\Gamma \in \mathbb{R}$ is a parameter that controls the stranght of transverse field.

# Futher Reading

- Any textbook for statistical mechanic (ex. Luca Peliti: Statistical Mechanics in a Nutshell, R.J. Baxter: Exactly Solved Models in Statistical Mechanics, H.E. Stanley: Introduction to Phase Transitions and Critical Phenomena etc.)
- Cipra, B. A. (1987). An introduction to the Ising model. *The American Mathematical Monthly*, 94(10), 937-959.
- Stinchcombe, R. B. (1973). Ising model in a transverse field. I. Basic theory. *Journal of Physics C: Solid State Physics*, 6(15), 2459.
- Barahona, F. (1982). On the computational complexity of Ising spin glass models. *Journal of Physics A: Mathematical and General*, 15(10), 3241.

